In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn import preprocessing
from sklearn.model_selection import KFold
from collections import deque
from datetime import datetime
import random
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from pickle import dump
# !pip install smogn shap tensorflow==2.5.0
# import smogn

drive_path = f"/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/megaset_today.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

NUM_FOLDS = 5  # for 10 we got val (input[test] of 10-fold) acc ~0.70 and test acc ~0.52. Overfiting to "val" set. Use 5, so its like 20% used for val
SEQ_LEN = 28
FUTURE_PERIOD_PREDICT = 1
EPOCHS = 500
BATCH_SIZE = 64
current_time = datetime.utcnow().strftime('%Y-%m-%d')
NAME = f"SAHMETEOautoenc{current_time}"

acc_per_fold = []
loss_per_fold = []
kfold = KFold(n_splits=NUM_FOLDS, shuffle=True)

# in k-fold we dont need validation set
def split_main_df(main_df, train_split, test_split):
    # [train train train train_index val val val val_index test test test_index]
    train_index = int(train_split * len(main_df))  # last indexes
    test_index = test_split  # we don't really need to use it, its always the last index

    train_df = main_df.iloc[:train_index]
    test_df = main_df.iloc[train_index:]
    return train_df, test_df

dataset = drive_path
df = pd.read_csv(dataset, nrows=None, skiprows=None, parse_dates=['DATA'])  # todo
# df.set_index("DATA", inplace=True)
daty = df['DATA']
df = df.drop(['DATA'], axis=1)
print(list(df.columns))
pd.crosstab(index=df['SAH'], columns='count')

df['SAH'] = np.where(df['SAH'] > 0, 1, 0)
pd.crosstab(index=df['SAH'], columns='count')

df

x = df.copy()
x = x.drop('SAH', axis=1)

y = df['SAH']

df

def temporalize(X, y, lookback):
    '''
    Inputs
    X         A 2D numpy array ordered by time of shape: 
              (n_observations x n_features)
    y         A 1D numpy array with indexes aligned with 
              X, i.e. y[i] should correspond to X[i]. 
              Shape: n_observations.
    lookback  The window size to look back in the past 
              records. Shape: a scalar.

    Output
    output_X  A 3D numpy array of shape: 
              ((n_observations-lookback-1) x lookback x 
              n_features)
    output_y  A 1D array of shape: 
              (n_observations-lookback-1), aligned with X.
    '''
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        output_y.append(y[i + lookback + 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

input_X = df.loc[:, df.columns != 'SAH'].values  # converts the df to a numpy array
input_y = df['SAH'].values

n_features = input_X.shape[1]  # number of features

def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize
    
    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [ ]:
input_X.shape

In [ ]:
while True:
  # RANDOM NUMBERS
  timestamp = str(time.time()) + "_complexLSTM_today_"
  lookback = random.randint(3, 30)
  epochs = random.randint(20, 100)
  batch = random.randint(24, 64)
  lstm_num = random.randint(6, 256)
  dropout = random.choice([0, 0.05, 0.1, 0.2])
  # cudnnlstm_num = random.randint(8, 128)
  lr = random.choice([0.0001,0.00001,0.001,0.01])
  no_selected_features = random.randint(5, input_X.shape[1])-1
  selected_features = random.sample(range(0,(input_X.shape[1]-1)), no_selected_features)


  from numpy.random import seed
  SEED = random.randint(1, 100) #used to help randomly select the data points
  seed(SEED)
  import tensorflow
  tensorflow.random.set_seed(SEED)
  # DATA_SPLIT_PCT = 0.15
  n_features = no_selected_features



  # Temporalize the data
  X, y = temporalize(X = input_X[:,selected_features], y = input_y, lookback = lookback)
  %matplotlib inline
  import matplotlib.pyplot as plt
  import seaborn as sns

  import pandas as pd
  import numpy as np
  from pylab import rcParams

  import tensorflow as tf
  from keras import optimizers, Sequential
  from keras.models import Model
  from keras.utils.vis_utils import plot_model
  from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
  from keras.callbacks import ModelCheckpoint, TensorBoard

  from sklearn.preprocessing import StandardScaler
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import confusion_matrix, precision_recall_curve
  from sklearn.metrics import recall_score, classification_report, auc, roc_curve
  from sklearn.metrics import precision_recall_fscore_support, f1_score



  LABELS = ["No","SAH"]
  rcParams['figure.figsize'] = 8, 6

  X_train = np.array(X)
  y_train = np.array(y)

  X_train_y0 = X_train[y_train==0]
  X_train_y1 = X_train[y_train==1]


  X_train = X_train.reshape(X_train.shape[0], lookback, n_features)
  X_train_y0 = X_train_y0.reshape(X_train_y0.shape[0], lookback, n_features)
  X_train_y1 = X_train_y1.reshape(X_train_y1.shape[0], lookback, n_features)

  # Initialize a scaler using the training data.
  scaler = StandardScaler().fit(flatten(X_train_y0))
  X_train_y0_scaled = scale(X_train_y0, scaler)
  a = flatten(X_train_y0_scaled)
  print('colwise mean', np.mean(a, axis=0).round(6))
  print('colwise variance', np.var(a, axis=0))

  X_train_scaled = scale(X_train, scaler)

  timesteps =  X_train_y0_scaled.shape[1] # equal to the lookback
  n_features =  X_train_y0_scaled.shape[2] # 59


  lstm_autoencoder = Sequential()
  # Encoder
  lstm_autoencoder.add(LSTM(lstm_num, input_shape=(timesteps, n_features), return_sequences=True))
  lstm_autoencoder.add(Dropout(rate=dropout))
  #lstm_autoencoder.add(CuDNNLSTM(cudnnlstm_num, return_sequences=False))
  lstm_autoencoder.add(LSTM(lstm_num, return_sequences=False))
  lstm_autoencoder.add(RepeatVector(timesteps))
  # Decoder
  #lstm_autoencoder.add(CuDNNLSTM(cudnnlstm_num, return_sequences=True))
  lstm_autoencoder.add(LSTM(lstm_num, return_sequences=True))
  lstm_autoencoder.add(Dropout(rate=dropout))
  lstm_autoencoder.add(LSTM(lstm_num, return_sequences=True))
  lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

  # lstm_autoencoder.build()

  adam = tf.optimizers.Adam(lr)
  lstm_autoencoder.compile(loss='mse', optimizer=adam)
  lstm_autoencoder.summary()

  cp = ModelCheckpoint(filepath=str(timestamp) + "lstm_autoencoder_classifier.h5",
                                save_best_only=True,
                                verbose=0)

  tb = TensorBoard(log_dir='./logs',
                  histogram_freq=0,
                  write_graph=True,
                  write_images=True)

  lstm_autoencoder_history = lstm_autoencoder.fit(X_train_y0_scaled, X_train_y0_scaled, 
                                                  epochs=epochs, 
                                                  batch_size=batch, 
                                                  callbacks=[keras.callbacks.EarlyStopping(monitor="loss", patience=25, mode="min")],
                                                  verbose=2).history

  train_x_predictions = lstm_autoencoder.predict(X_train_scaled)

  t_mse = np.mean(np.power(flatten(X_train_scaled) - flatten(train_x_predictions), 2), axis=1)

  t_error_df = pd.DataFrame({'Reconstruction_error': t_mse,
                          'True_class': y_train.tolist()})

  t_false_pos_rate, t_true_pos_rate, thresholds = roc_curve(t_error_df.True_class, t_error_df.Reconstruction_error)
  t_roc_auc = auc(t_false_pos_rate, t_true_pos_rate,)
  print("TRAINING ROC AUC = " + str(t_roc_auc))

  def Find_Optimal_Cutoff(target, predicted):
      """ Find the optimal probability cutoff point for a classification model related to event rate
      Parameters
      ----------
      target : Matrix with dependent or target data, where rows are observations

      predicted : Matrix with predicted data, where rows are observations

      Returns
      -------     
      list type, with optimal cutoff value
          
      """
      fpr, tpr, threshold = roc_curve(target, predicted)
      i = np.arange(len(tpr)) 
      roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
      roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

      return list(roc_t['threshold']) 

  if t_roc_auc>0.70:
    print("Model worth saving.")
    threshold = Find_Optimal_Cutoff(y_train, t_mse)
    print(threshold)

    threshold_fixed = threshold
    


    # Training set
    groups = t_error_df.groupby('True_class')
    fig, ax = plt.subplots()

    for name, group in groups:
        ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
                label= "SAH" if name == 1 else "No SAH")
    ax.hlines(threshold_fixed, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error for different classes")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    plt.savefig("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'training-error-threshold.png')
    plt.show()

    pred_y = [1 if e > threshold_fixed else 0 for e in t_error_df.Reconstruction_error.values]
    conf_matrix = confusion_matrix(t_error_df.True_class, pred_y)

    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.savefig("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'training-cm.png')
    plt.show()

    t_accuracy1=(conf_matrix[0,0]+conf_matrix[1,1])/(conf_matrix[0,0]+conf_matrix[0,1]+conf_matrix[1,0]+conf_matrix[1,1])
    print ('Training Accuracy : ', t_accuracy1)

    t_sensitivity1 = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    print('Training Sensitivity : ', t_sensitivity1 )

    t_specificity1 = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1])
    print('Training Specificity : ', t_specificity1)



    plt.plot(t_false_pos_rate, t_true_pos_rate, linewidth=5, label='AUC = %0.3f'% t_roc_auc)
    plt.plot([0,1],[0,1], linewidth=5)

    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.legend(loc='lower right')
    plt.title('Receiver operating characteristic curve (ROC)')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'training-roc.png')
    plt.show()

    plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
    plt.legend(loc='upper right')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.savefig("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'training-history.png')
    plt.show()

    precision_rt, recall_rt, threshold_rt = precision_recall_curve(t_error_df.True_class, t_error_df.Reconstruction_error)
    plt.plot(threshold_rt, precision_rt[1:], label="Precision",linewidth=5)
    plt.plot(threshold_rt, recall_rt[1:], label="Recall",linewidth=5)
    plt.title('Precision and recall for different threshold values')
    plt.xlabel('Threshold')
    plt.ylabel('Precision/Recall')
    plt.legend()
    plt.savefig("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'training-precision-recall.png')
    plt.show()

    daty_temporalized, y_temporalized = temporalize(X = np.transpose(np.array([range(0, daty.shape[0]) , daty.astype(str)])), y = input_y, lookback = lookback)
    uzyte_daty = []
    for i in range(len(daty_temporalized)):
        uzyte_daty.append(daty_temporalized[i][daty_temporalized.shape[1]-1][1])
    error_df = pd.DataFrame({'Data': uzyte_daty, 'Reconstruction_error': t_mse,
                          'True_class': y_train.tolist()})
    error_df.to_csv("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'error_df.csv')

    # Save model
    dump(scaler, open("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) +  'scaler.pkl', 'wb'))
    lstm_autoencoder.save("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'model.h5')

    stats = pd.DataFrame({'timestamp': timestamp, 'lookback': lookback, 'epochs': epochs, 'batch':batch, 'lstm':lstm_num, 'dropout':dropout, 'lr':lr, 'threshold':threshold, 't_roc_auc':t_roc_auc, 't_accuracy':t_accuracy1, 't_sensitivity':t_sensitivity1, 't_specificity':t_specificity1, 'no_selected_features':no_selected_features, 'selected_features':str(selected_features), 'features':'+'.join(map(str,df.columns[selected_features])), 'seed':SEED})
    stats.to_csv("/content/drive/MyDrive/Projekty/SAHMeteo/LSTM/COMBO/" + str(timestamp) + 'stats.csv')